# Brightfield Segmentation
This notebook needs to be run in an environment which has cellpose installed alongside the correct CUDA and pytorch version for the processing to work in a fast manner. First, we need to load the images and do some normalisation:

In [1]:
import numpy as np
import skimage.io as io
import os
from cellpose import models
from cellpose.io import imread

# needs to be changed to repository directory
home_directory = r"C:\Users\savill\OneDrive\Documents\PhD Jesse\Embryonic_organoid_prediction\Processing dataset for Paper\TLS_2D_morphostate_investigation/"
folder = home_directory + r"image_data\Processed Data\Focussed Images\48h\Brightfield"

image_collection = io.imread_collection(folder+"/*.tif")
file_names = [os.path.split(fname)[1] for fname in image_collection.files]

def min_max_scaling(img):
    return (img-np.min(img))/(np.max(img)-np.min(img))

scaled = [min_max_scaling(img) for img in image_collection]
print(f"shape: {len(scaled)}, {scaled[0].shape}")  

shape: 768, (598, 712)


In [2]:
from cellpose import core
use_GPU = core.use_gpu()
use_GPU

True

After making sure that we are using the GPU we can load the model and segment the images

In [3]:
%%time
path_to_model = home_directory + r"models\CP_20230302_brightfield_48h"
model = models.CellposeModel(gpu=use_GPU,pretrained_model=path_to_model)
channels = [[0,0]]
diameter = 100

masks, foo, bar = model.eval(scaled, diameter=diameter, channels=channels)

CPU times: total: 10min 8s
Wall time: 11min 58s


In [4]:
len(masks)

768

Once the segmentation is performed we can save the output:

In [5]:
output = home_directory + r"image_data\Processed Data\Masks Brightfield\48h"

if not os.path.exists(output):
   os.makedirs(output)
   print("The new directory is created!")

for name, mask in zip(file_names,masks):
    name_mask = f"{name[:-4]}mask.tif"
    io.imsave(f"{output}/{name_mask}",mask.astype("uint8"))

C:\Users\savill\AppData\Local\Temp\ipykernel_7700\727197146.py:9: UserWarning: C:\Users\savill\OneDrive\Documents\PhD Jesse\Embryonic_organoid_prediction\Processing dataset for Paper\TLS_2D_morphostate_investigation/image_data\Processed Data\Masks Brightfield\48h/FK223_run_RR_PLATE_1_ID_A1_C_BTFmask.tif is a low contrast image
  io.imsave(f"{output}/{name_mask}",mask.astype("uint8"))
C:\Users\savill\AppData\Local\Temp\ipykernel_7700\727197146.py:9: UserWarning: C:\Users\savill\OneDrive\Documents\PhD Jesse\Embryonic_organoid_prediction\Processing dataset for Paper\TLS_2D_morphostate_investigation/image_data\Processed Data\Masks Brightfield\48h/FK223_run_RR_PLATE_1_ID_A2_C_BTFmask.tif is a low contrast image
  io.imsave(f"{output}/{name_mask}",mask.astype("uint8"))
C:\Users\savill\AppData\Local\Temp\ipykernel_7700\727197146.py:9: UserWarning: C:\Users\savill\OneDrive\Documents\PhD Jesse\Embryonic_organoid_prediction\Processing dataset for Paper\TLS_2D_morphostate_investigation/image_data